In [99]:
import os
import openai
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader, PyPDFLoader
from langchain.text_splitter import NLTKTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores.faiss import FAISS
from IPython.display import display, Markdown

### Deploy the LLM
You can change the model with OpenAI, Gemini, Llama, etc.

In [2]:
# API key for LLM
AZURE_OPENAI_KEY = "xxxxxxxxxxxxxxxxxxxx" # change it with your own key
os.environ['AZURE_OPENAI_KEY'] = AZURE_OPENAI_KEY
openai.api_version = "202x-xx-xx-preview" # change it with your own version
openai.azure_endpoint = "https://xxxxx.openai.azure.com/" # change it with your own endpoint

model = "gpt35turbo" # deployment name on Azure OPENAI Studio
myLLM = AzureChatOpenAI(azure_endpoint = openai.azure_endpoint, 
                            api_key=AZURE_OPENAI_KEY,  
                            api_version=openai.api_version,
                            temperature=0,
                            streaming=True, 
                            model = model,)

### Deploy the Parser
You can change the PDF Loader with PyPDFLoader, LlamaParse, etc.

Car Manuals: https://www.toyota.co.uk/customer/manuals

In [6]:
# API key for loader
key = "xxxxxxxxxxxxxxxxxxxx" # change it with your own loader key
endpoint = "https://xxxxxx.azure.com/" # change it with your own loader endpoint

# PDF Loader
path = "xxx.pdf" # path of a manual that we used for RAG mode
myPDFLoader = AzureAIDocumentIntelligenceLoader(
        api_endpoint=endpoint, 
        api_key=key, 
        file_path=path, 
        api_model="prebuilt-layout",
        mode="page"
    )

### Deploy the Embedding and Splitter
You can also change the embedding model with OpenAI, Gecko, etc.

Embedding Benchmark Table: https://huggingface.co/spaces/mteb/leaderboard

In [17]:
# Embedding Model
embedding_name = "embedding3large" # deployment name on Azure OPENAI Studio
myEmbeddingModel = AzureOpenAIEmbeddings(azure_deployment=embedding_name,
        azure_endpoint = openai.azure_endpoint, 
        api_key=AZURE_OPENAI_KEY,  
        api_version=openai.api_version)

# Splitter
chunk_size = 1000
chunk_overlap = 200
language = "english"
mySplitter = NLTKTextSplitter(chunk_size=chunk_size, 
                              chunk_overlap=chunk_overlap, 
                              add_start_index=True,
                              language=language)

### Load and Split the Document

In [12]:
documents = myPDFLoader.load()
print("Number of chunks before split: ", len(documents))
documents[0:5]

Number of chunks before split:  712


[Document(page_content='1\n65432\n7\n8\nOwners Manual_Europe_M9A608_enPictorial index Search by illustration\nFor safety \nand securityMake sure to read through them\n(Main topics: Child seat, theft deterrent system)\nVehicle status information and indicatorsReading driving-related information\n(Main topics: Meters, multi-information display)\nBefore drivingOpening and closing the doors and windows, adjustment \nbefore driving\n(Main topics: Keys, doors, seats, power windows)\nDrivingOperations and advice which are necessary for driving\n(Main topics: Starting hybrid system, refueling)\nInterior featuresUsage of the interior features\n(Main topics: Air conditioner, storage features)\nMaintenance \nand careCaring for your vehicle and maintenance procedures\n(Main topics: Interior and exterior, light bulbs)\nWhen trouble arises What to do in case of malfunction and emergency\n(Main topics: 12-volt battery discharge, flat tire)\nVehicle specifications Vehicle specifications, customizable 

In [19]:
chunks = mySplitter.split_documents(documents)
print("Number of chunks after split: ", len(chunks))
chunks[0:5]

Number of chunks after split:  1457


[Document(page_content='1\n65432\n7\n8\nOwners Manual_Europe_M9A608_enPictorial index Search by illustration\nFor safety \nand securityMake sure to read through them\n(Main topics: Child seat, theft deterrent system)\nVehicle status information and indicatorsReading driving-related information\n(Main topics: Meters, multi-information display)\nBefore drivingOpening and closing the doors and windows, adjustment \nbefore driving\n(Main topics: Keys, doors, seats, power windows)\nDrivingOperations and advice which are necessary for driving\n(Main topics: Starting hybrid system, refueling)\nInterior featuresUsage of the interior features\n(Main topics: Air conditioner, storage features)\nMaintenance \nand careCaring for your vehicle and maintenance procedures\n(Main topics: Interior and exterior, light bulbs)\nWhen trouble arises What to do in case of malfunction and emergency\n(Main topics: 12-volt battery discharge, flat tire)\nVehicle specifications Vehicle specifications, customizable 

### Create the Database and Save the Embedded Document

In [29]:
def save_database(chunks, embeddingModel, name):
    search_index = FAISS.from_documents(chunks, embeddingModel)

    search_index.save_local(name)
    
def load_database(embeddingModel, name):
    db = FAISS.load_local(name, embeddingModel, allow_dangerous_deserialization=True)
    
    return db

name = path.split("/")[-1].split(".")[0] + "-vector-db" # name of the database
save_database(chunks=chunks, embeddingModel=myEmbeddingModel, name=name)


In [30]:
# Once you save embedded documents to a database, you can load it after
db = load_database(embeddingModel=myEmbeddingModel, name=name)

### Deploy the retriever
Document for search type: https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.as_retriever


In [31]:
retriever = db.as_retriever(search_type="mmr")

### Create a Memory
It keeps the history of the chat to ask relevent questions with previous ones.

In [33]:
memory = ConversationBufferMemory(memory_key="history",
                                  input_key="question")


### Create the Chat Template and the Prompt

In [35]:
template = """
    You are a helpful service bot that provides people with information about their vehicles. 
    When answering questions in <<<<>>>, you will use only the information in the context, without using your prior knowledge. 
    Before answering, evaluate the relationship between context and question and don't use context that you don't want to include.
    If the question is not in the context, you don't have to answer it.
    
    # Context: {context}
    
    # History: {history}
    
    <<<
    Question: {question}
    >>>
    
    # Answer:
    """
    
PROMPT = ChatPromptTemplate.from_template(template)

### Deploy the RAG Model

In [36]:
myRAGModel = RetrievalQA.from_chain_type(
                llm=myLLM,
                chain_type="stuff",
                retriever=retriever,
                chain_type_kwargs={"prompt": PROMPT,
                                   "memory": memory},
                return_source_documents=True,
                )

In [69]:
# to print the result
def print_result(query, result):
  output_text = f"""### Question: 
  {query}
  ### Answer: 
  {result['result']}
  ### All relevant pages:
  {', '.join(list(set([str(doc.metadata['page']) for doc in result['source_documents']])))}
  """
  return(output_text)

### Run the RAG Model

In [95]:
q1 = "My car is dripping something dark on the floor, what that it means?"
a1 = myRAGModel.invoke(q1)

display(Markdown(print_result(q1, a1)))

# you can see the source documents in a1

### Question: 
  My car is dripping something dark on the floor, what that it means?
  ### Answer: 
  If your car is dripping something dark on the floor, it could indicate an oil leak. It is important to have your vehicle checked by a professional to determine the source of the leak and to prevent any further damage.
  ### All relevant pages:
  422, 410, 414, 531
  

In [96]:
q2 = "Where could be the source?"
a2 = myRAGModel.invoke(q2)

display(Markdown(print_result(q2, a2)))

### Question: 
  Where could be the source?
  ### Answer: 
  The source of the oil leak could be various components in the vehicle's engine or transmission system. It is recommended to have your vehicle checked by a professional to accurately determine the source of the leak.
  ### All relevant pages:
  299, 9, 424, 153
  

In [97]:
# to clear the memory of the conversation
memory.clear()